# initialization

In [5]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.ERROR)

In [6]:
%run /home/sparkuser/ipython/native_sql_initialize.ipynb

,Variable,Value
0,SHELL,/bin/bash
1,SUDO_GID,100
2,JAVA_HOME,/usr/lib/jvm/java-8-openjdk-amd64
3,SUDO_COMMAND,/bin/bash
4,SUDO_USER,binweiyang
5,PWD,/home/sparkuser/ipython
6,LOGNAME,sparkuser
7,HOME,/home/sparkuser
8,YARN_CONF_DIR,/home/sparkuser/hadoop/etc/hadoop
9,LANG,C.UTF-8


Spark version: 3.3.2
localhost: devrestricted-binweiyang
local_ip: 10.9.235.144
proxy: http://10.239.44.250:8080
spark.eventLog.dir: hdfs://devrestricted-binweiyang:8020/tmp/sparkEventLog


In [7]:
codec='zstd'
tabledir='/tpcds_sf1000_zstd/'
workload='tpcds'
run_gluten=True

In [8]:
# Fixed parameters. Need to be modified manually when migrating to new machine.
clients=[localhost]

executors_per_node=2
cores_per_executor=8
memory_per_node='60g'

disk_dev=['dev259-0']

collect_emon=False

master=f'spark://{localhost}:7077'

In [9]:
workload="tpch"

In [11]:
# Fixed parameters decided by input params
jars = ''
nodes=len(clients)

appname=''
is_tpch_workload=False
is_tpcds_workload=False

if workload.lower() == 'tpch':
    app_name = 'tpch_power'
    is_tpch_workload=True
elif workload.lower() == 'tpcds':
    app_name = 'tpcds_power'
    is_tpcds_workload=True
else:
    raise ValueError(f"Unknown workload: {workload}")

if run_gluten:
    target_jar = f'/home/binweiyang/gluten-velox-bundle-spark3.3_2.12-ubuntu_20.04-1.1.0-SNAPSHOT.jar'

    !ls -l {target_jar}

    jars_base="/home/sparkuser/jars/spark33"
    !mkdir -p {jars_base}
    !rm -rf {jars_base}/*
    !cp {target_jar} {jars_base}/
    jars=!ls -d {jars_base}/*.jar
    jars=":".join(["file://"+j for j in jars])
    print(jars)

    task_per_core=3
    app_name = ' '.join(['gluten', app_name])
else:
    task_per_core=8
    app_name = ' '.join(['vanilla', app_name])

-rw-r--r-- 1 binweiyang users 81846284 Sep 24 07:58 /home/binweiyang/gluten-velox-bundle-spark3.3_2.12-ubuntu_20.04-1.1.0-SNAPSHOT.jar
file:///home/sparkuser/jars/spark33/gluten-velox-bundle-spark3.3_2.12-ubuntu_20.04-1.1.0-SNAPSHOT.jar


In [17]:
!ls /home/sparkuser/jars/spark33/gluten-velox-bundle-spark3.3_2.12-ubuntu_20.04-1.1.0-SNAPSHOT.jar

/home/sparkuser/jars/spark33/gluten-velox-bundle-spark3.3_2.12-ubuntu_20.04-1.1.0-SNAPSHOT.jar


# Run Workload

In [13]:
def global_conf_overwrite(conf, workload_conf_overwrite=None):
    if workload_conf_overwrite:
        conf = workload_conf_overwrite(conf)
#     conf.set(xxx)
    return conf

In [14]:
def gluten_tpcds_conf_overwrite(conf):
    conf.set('spark.sql.optimizer.runtime.bloomFilter.applicationSideScanSizeThreshold', '0')\
        .set('spark.sql.optimizer.runtime.bloomFilter.enabled', 'true')\
        .set('spark.gluten.sql.columnar.joinOptimizationLevel', '18')\
        .set('spark.gluten.sql.columnar.physicalJoinOptimizeEnable', 'true')\
        .set('spark.gluten.sql.columnar.physicalJoinOptimizationLevel', '18')\
        .set('spark.gluten.sql.columnar.logicalJoinOptimizeEnable', 'true')\
        .set('spark.gluten.sql.columnar.logicalJoinOptimizationLevel', '19')\
        .set('spark.gluten.sql.columnar.shuffle.codec', codec) \
        .set('spark.gluten.sql.columnar.backend.velox.memoryCapRatio',0.75)
    return conf

def vanilla_tpcds_conf_overwrite(conf):
    conf.set('spark.sql.optimizer.runtime.bloomFilter.applicationSideScanSizeThreshold', '0')\
        .set('spark.sql.optimizer.runtime.bloomFilter.enabled', 'true')\
        .set('spark.io.compression.codec', codec)
    return conf

In [15]:
def gluten_tpch_conf_overwrite(conf):
    conf.set('spark.gluten.sql.columnar.shuffle.codec', codec)
    return conf

def vanilla_tpch_conf_overwrite(conf):
    conf.set('spark.io.compression.codec', codec)
    return conf

In [20]:
create_cntx_func

<function __main__.create_cntx_gluten(executors_per_node, cores_per_executor, task_per_core, memory_per_node, extra_jars, app_name='', master='yarn', conf_overwrite=None)>

In [19]:
# Create SparkSession

workload_conf_overwrite=None
create_cntx_func=None

if run_gluten:
    create_cntx_func=create_cntx_gluten
    if is_tpcds_workload:
        workload_conf_overwrite = gluten_tpcds_conf_overwrite
    else:
        workload_conf_overwrite = gluten_tpch_conf_overwrite
else:
    create_cntx_func=create_cntx_vanilla
    if is_tpcds_workload:
        workload_conf_overwrite = vanilla_tpcds_conf_overwrite
    else:
        workload_conf_overwrite = vanilla_tpch_conf_overwrite

conf_overwrite = lambda conf: global_conf_overwrite(conf, workload_conf_overwrite)
sc, spark=create_cntx_func(executors_per_node, cores_per_executor, task_per_core, memory_per_node, jars, app_name, master, conf_overwrite)


        executors per node: 2
        parallelism: 48
        executor memory: 1558m
        offheap memory: 10907m
    
spark.serializer:  org.apache.spark.serializer.KryoSerializer
master:  spark://devrestricted-binweiyang:7077


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.ClassNotFoundException: org.apache.spark.shuffle.sort.ColumnarShuffleManager
	at java.net.URLClassLoader.findClass(URLClassLoader.java:387)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:418)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:351)
	at java.lang.Class.forName0(Native Method)
	at java.lang.Class.forName(Class.java:348)
	at org.apache.spark.util.Utils$.classForName(Utils.scala:218)
	at org.apache.spark.util.Utils$.instantiateSerializerOrShuffleManager(Utils.scala:2693)
	at org.apache.spark.SparkEnv$.create(SparkEnv.scala:318)
	at org.apache.spark.SparkEnv$.createDriverEnv(SparkEnv.scala:194)
	at org.apache.spark.SparkContext.createSparkEnv(SparkContext.scala:279)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:464)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at sun.reflect.NativeConstructorAccessorImpl.newIns

In [ ]:
if run_gluten:
    pinexecutor_numa(clients)

In [ ]:
appid = sc.applicationId
print("start run: ", appid)

if is_tpch_workload:
    test_tpc = TestTPCH(spark, tabledir,'parquet')
elif is_tpcds_workload:
    test_tpc = TestTPCDS(spark, tabledir, 'parquet')
else:
    raise ValueError(f"Unknown workload: {workload}")

test_tpc.clear_result()

In [ ]:
startmonitor(clients, appid, collect_emon=collect_emon)

In [ ]:
test_tpc.load_all_tables_as_tempview()
for l in test_tpc.query_ids:
    x=test_tpc.run_query(l, explain = False)

In [ ]:
stopmonitor(clients, sc, appid, upload_sr525=False)

In [ ]:
sc.stop()

# Show Performance

In [2]:
draw_sar(appid, qtime=test_tpc.collect_result(), disk_dev=disk_dev)

NameError: name 'draw_sar' is not defined

In [ ]:
test_tpc.print_result()